<a href="https://colab.research.google.com/github/skywalker0803r/Ricky/blob/master/%E7%8E%89%E5%B1%B1AI%E6%AF%94%E8%B3%BD_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import warnings 
warnings.simplefilter('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# train

In [2]:
import pandas as pd
train = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/train_預處理完.csv",index_col=0)
print(train.shape)
train.head()

(1065250, 53)


,acqic,bacno,cano,conam,csmcu,ecfg,etymd,insfg,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,txkey,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,flbmk_N,flbmk_Y,flbmk_unknown,flg_3dsmk_N,flg_3dsmk_Y,flg_3dsmk_unknown,hcefg_0,hcefg_1,hcefg_2,hcefg_3,hcefg_5,hcefg_6,hcefg_7,hcefg_8,hcefg_9,iterm_0,iterm_1,iterm_2,iterm_3,iterm_4,iterm_5,iterm_6,iterm_7,iterm_8,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4,fraud_ind
794747,6557,51892,58488,497.33,0.007184,0,0.000562,0,50,210608.0,0.000794,31635,0,5817,0.000914,1649054,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
915118,5975,65754,58218,718.77,0.007184,0,0.000562,0,85,120312.0,0.000570,99026,0,5817,0.000914,1456457,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
553093,6716,163544,117141,807.34,0.007184,1,0.020633,0,6,180139.0,0.000780,34227,0,5820,0.000914,1550968,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
525146,6716,52788,8114,1110.00,0.007184,0,0.020633,0,5,95739.0,0.001960,35002,0,5817,0.000914,1121906,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
709211,6716,62527,147619,669.38,0.007184,0,0.000488,0,25,155732.0,0.001135,48833,0,5859,0.000914,1206078,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0


# val

In [3]:
val = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/val_預處理完.csv",index_col=0)
print(val.shape)
val.head()

(456537, 53)


,acqic,bacno,cano,conam,csmcu,ecfg,etymd,insfg,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,txkey,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,flbmk_N,flbmk_Y,flbmk_unknown,flg_3dsmk_N,flg_3dsmk_Y,flg_3dsmk_unknown,hcefg_0,hcefg_1,hcefg_2,hcefg_3,hcefg_5,hcefg_6,hcefg_7,hcefg_8,hcefg_9,iterm_0,iterm_1,iterm_2,iterm_3,iterm_4,iterm_5,iterm_6,iterm_7,iterm_8,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4,fraud_ind
1195536,6716,20696,36558,387.93,0.007184,0,0.000488,0,40,202323.0,0.000540,35952,0,5820,0.000914,1143723,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
427655,3348,63793,134942,941.38,0.007184,1,0.047257,0,51,110732.0,0.080097,54238,0,6580,0.089083,724559,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
579151,5975,58864,177484,545.53,0.007184,0,0.020633,0,82,151032.0,0.000151,96744,0,5817,0.000914,1312059,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
221857,6581,65275,85515,245.29,0.007184,0,0.006002,0,85,81533.0,0.000151,5546,0,5812,0.000914,1670608,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
460164,5975,46343,147035,689.60,0.007184,0,0.000488,0,14,214624.0,0.001071,97673,0,5817,0.000914,1770025,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0


# test

In [4]:
test = pd.read_csv("/content/drive/My Drive/玉山人工智慧比賽數據/test_預處理完.csv",index_col=0)
test_txkey = test["txkey"]
print(test.shape)
test.head()

(421665, 52)


,acqic,bacno,cano,conam,csmcu,ecfg,etymd,insfg,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,txkey,contp_0,contp_1,contp_2,contp_3,contp_4,contp_5,contp_6,flbmk_N,flbmk_Y,flbmk_unknown,flg_3dsmk_N,flg_3dsmk_Y,flg_3dsmk_unknown,hcefg_0,hcefg_1,hcefg_2,hcefg_3,hcefg_5,hcefg_6,hcefg_7,hcefg_8,hcefg_9,iterm_0,iterm_1,iterm_2,iterm_3,iterm_4,iterm_5,iterm_6,iterm_7,iterm_8,stscd_0,stscd_1,stscd_2,stscd_3,stscd_4
0,6881,163188,116799,513.8,0.001086,0,0.001428,0,102,215328.0,0.001712,59360,0,0,0.000914,592489,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
1,6881,163188,116799,513.8,0.001086,0,0.001428,0,102,222007.0,0.001712,59360,0,0,0.000914,592452,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
2,6881,163188,116799,513.8,0.001086,0,0.001428,0,100,170013.0,0.001712,59360,0,0,0.000914,590212,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
3,6881,163188,116799,513.8,0.001086,0,0.001428,0,100,165914.0,0.001712,59360,0,0,0.000914,590209,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0
4,6881,163188,116799,513.8,0.001086,0,0.001428,0,102,215311.0,0.001712,59360,0,0,0.000914,592488,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0


# 定義 features & target

In [5]:
features = test.drop(['txkey'],axis=1).columns.tolist()
y_name = 'fraud_ind'
print(features)
print([y_name])

['acqic', 'bacno', 'cano', 'conam', 'csmcu', 'ecfg', 'etymd', 'insfg', 'locdt', 'loctm', 'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'contp_0', 'contp_1', 'contp_2', 'contp_3', 'contp_4', 'contp_5', 'contp_6', 'flbmk_N', 'flbmk_Y', 'flbmk_unknown', 'flg_3dsmk_N', 'flg_3dsmk_Y', 'flg_3dsmk_unknown', 'hcefg_0', 'hcefg_1', 'hcefg_2', 'hcefg_3', 'hcefg_5', 'hcefg_6', 'hcefg_7', 'hcefg_8', 'hcefg_9', 'iterm_0', 'iterm_1', 'iterm_2', 'iterm_3', 'iterm_4', 'iterm_5', 'iterm_6', 'iterm_7', 'iterm_8', 'stscd_0', 'stscd_1', 'stscd_2', 'stscd_3', 'stscd_4']
['fraud_ind']


# modeling

In [6]:
cat_indices = []
for i,col in enumerate(features):
  u = len(train[col].unique())
  if u != 0:
    cat_indices.append(i)
print(cat_indices)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


In [7]:
import lightgbm as lgb
model = lgb.LGBMClassifier(n_jobs = -1)
model.fit(train[features],train[y_name],categorical_feature = cat_indices)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# F1 report

In [8]:
from sklearn.metrics import classification_report 

y_pred = model.predict(val[features])
print(classification_report(val[y_name] ,y_pred ,target_names=['0','1']))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00    450415
           1       0.82      0.60      0.69      6122

    accuracy                           0.99    456537
   macro avg       0.91      0.80      0.85    456537
weighted avg       0.99      0.99      0.99    456537



# 將val和train合併後重新train model

In [9]:
model = lgb.LGBMClassifier(n_jobs = -1)
model.fit(train[features].append(val[features]),
          train[y_name].append(val[y_name]),
          categorical_feature = cat_indices)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

# 產生submission

In [10]:
submission = pd.DataFrame()
submission["txkey"] = test_txkey
submission["fraud_ind"] = model.predict(test[features])

print(submission["fraud_ind"].value_counts())
submission.head()

0    416927
1      4738
Name: fraud_ind, dtype: int64


,txkey,fraud_ind
0,592489,0
1,592452,0
2,590212,0
3,590209,0
4,592488,0


# 保存

In [0]:
submission.to_csv("./submission_17.csv")